# Churn 2.0 Project

In [1]:
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns 
#sns.set_theme(style="whitegrid")
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import time
import pickle
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery_storage_v1
import string
import xgboost as xgb

import churn_preprocessing_v5 as cpp
import churn_modeling as cm

In [2]:
# Set up the connection to GBG
bqconn = cpp.bqConnect('alticeusa-am-b639e404289b.json', 'alticeusa-am')

# Get the master table
raw_data_sql = '''select * from eds_churn.t_ds_fixed_churn_score where month_id = '202202' and upper(cust_class_fs_type_desc) = 'RESIDENTIAL' and metro_area_id = 401'''
raw_data_res = bqconn.fetch_data(raw_data_sql)

raw_data_res.shape

(4925204, 340)

In [5]:
raw_data_res.head().T

,2,7,8,9,10
month_id,202207,202207,202207,202207,202207
curr_dt,2022-07-31,2022-07-31,2022-07-31,2022-07-31,2022-07-31
chc_id,787653587901,770765568801,780151673907,771029290901,770754668901
corp,7876,7707,7801,7710,7707
house,535879,655688,516739,292909,546689
cust,1,1,7,1,1
account_id,9947121,24530486,7424210,28003554,27398515
house_id,5578166,12000994,291560,10147726,9527685
fta,1,60,1,22,25
node_latitude,40.809242,-1.0,40.750956,-1.0,-1.0


## Extract only below useful columns for modelling

In [6]:
cat_cols = [
'head_end_desc', 'local_market_segment', 
'metro_area_desc', 'product_class_desc', 'cust_equip_class', 
'ecohort_code_desc', 'ecohort_income_range_desc', 'ecohort_age_range_desc', 'ecohort_spending_type_desc', 'ecohort_credit_line_type_desc', 'ecohort_credit_usage_type_desc', 'archetype_desc', 
'ebill_status_desc',
'agent_last_dept',   
'agent_repeat_1_prevDept', 'agent_repeat_1_prevInteractionReason', 'agent_repeat_2_prevDept', 
'agent_repeat_2_prevInteractionReason', 'agent_repeat_3_prevInteractionReason', 'agent_repeat_3_prevDept', 
'ivr_last_dept', 'ivr_last_interactionReason', 'ivr_last_ivrInteractionEndResult', 
'ivr_repeat_1_prevDept', 'ivr_repeat_1_prevInteractionReason',
'ivr_repeat_2_prevDept', 'ivr_repeat_2_prevInteractionReason',
'ivr_repeat_3_prevDept', 'ivr_repeat_3_prevInteractionReason',
'speed_m0', 'speed_m1', 'speed_m2', 'speed_m3', 'speed_m4', 
'curr_video_tier_desc_m0', 'curr_video_tier_desc_m1', 'curr_video_tier_desc_m2', 'curr_video_tier_desc_m3', 'curr_video_tier_desc_m4', 
'curr_ov_tier_desc_m0', 'curr_ov_tier_desc_m1', 'curr_ov_tier_desc_m2', 'curr_ov_tier_desc_m3', 'curr_ov_tier_desc_m4', 
'hbo_svod_new_m1', 'hbo_svod_new_m2', 'hbo_svod_new_m3',
'max_svod_m1', 'max_svod_m2', 'max_svod_m3', 
'ool_offer_group_v2_desc_m0', 'ool_offer_group_v2_desc_m1', 'ool_offer_group_v2_desc_m2', 'ool_offer_group_v2_desc_m3', 'ool_offer_group_v2_desc_m4',
'oolpromo_rng_m0',             
'ov_offer_group_v2_desc_m0', 'ov_offer_group_v2_desc_m1', 'ov_offer_group_v2_desc_m2', 'ov_offer_group_v2_desc_m3', 'ov_offer_group_v2_desc_m4',   
'ovpromo_rng_m0',            
'stz_enc_svod_m1', 'stz_enc_svod_m2', 'stz_enc_svod_m3', 
'video_offer_group_desc_m0', 'video_offer_group_desc_m1', 'video_offer_group_desc_m2', 'video_offer_group_desc_m3', 'video_offer_group_desc_m4', 
'service_visit_1_tc_problem_code_desc', 'service_visit_1_tc_fix_code_desc', 'service_visit_2_tc_problem_code_desc', 'service_visit_2_tc_fix_code_desc', 
'service_visit_3_tc_problem_code_desc', 'service_visit_3_tc_fix_code_desc', 
'videopromo_rng_m0']           


num_cols = [
'analog_revenue',  'iptel_revenue', 'ool_revenue',  'dvr_revenue',
'auto_ind', 
'agent_abandoned_standard_7day', 'agent_abandoned_7day', 'agent_offered_7day', 'agent_accepted_7day', 'agent_abandoned_standard_30day', 'agent_abandoned_30day', 'agent_offered_30day', 'agent_accepted_30day',
'agent_repeat_calls_30day', 'ivr_calls_7day', 'ivr_calls_30day', 'ivr_repeat_calls_30day', 
'service_visits_7day', 'service_visits_30day', 'service_visits_90day', 
'outages_90day', 'outages_60day', 'outages_30day', 
'rate_event_count_60day', 
'promo_rolloff_count_60day', 
'FAILSTAT_OFFLINEEVT', 'FAILSTAT_DHCP_MACADDR', 'FAILSTAT_WIFI_REBOOT_COUNT', 'FAILSTAT_WIFI_QOE_HOME_30',
'baserev_m0', 'baserev_m1', 'baserev_m2', 'baserev_m3', 'baserev_m4', 'baserev_m5', 'baserev_m6', 
'cust_svc_bill_agent_offered_m0', 'cust_svc_bill_agent_offered_m1', 'cust_svc_bill_agent_offered_m2', 'cust_svc_bill_agent_offered_m3', 'cust_svc_bill_agent_offered_m4', 'cust_svc_bill_agent_offered_m5', 'cust_svc_bill_agent_offered_m6', 
'tot_ool_gb_all_m1', 'tot_ool_gb_all_m2',
'roll_off_lift_m0' , 'roll_off_lift_m1', 'roll_off_lift_m2', 'roll_off_lift_m3', 'roll_off_lift_m4',              
'tot_retention_contacts_m0', 'tot_retention_contacts_m1', 'tot_retention_contacts_m2', 'tot_retention_contacts_m3', 'tot_retention_contacts_m4', 
'flag_altice_CABLE', 'flag_altice_DOCSIS_3_0', 'flag_altice_DOCSIS_3_1', 'flag_altice_DSL', 'flag_altice_FIBER', 'flag_altice_OTHER', 
'flag_altice_TERRESTRIAL_FIXED', 'flag_comp_CABLE', 'flag_comp_DOCSIS_3_0', 'flag_comp_DOCSIS_3_1', 'flag_comp_DSL', 'flag_comp_FIBER', 'flag_comp_OTHER', 'flag_comp_TERRESTRIAL_FIXED',
'cust_tenure_days'
]

extra_cols = ['chc_id']

print(len(cat_cols))
print(len(num_cols))

74
75


In [7]:
raw_data_res = raw_data_res[cat_cols + num_cols  + extra_cols]
print('shape : ', raw_data_res.shape)

raw_data_res.head(5)

(4542546, 150)

## Pre-Processing

In [21]:
raw_data_res_cpy = raw_data_res.copy()

### Step1 : Reduce levels in Categorical columns

In [27]:
raw_data_res_cpy[cat_cols] = raw_data_res_cpy[cat_cols].fillna(value='None')
dict1 = cpp.reduce_cat_cols_levels(raw_data_res_cpy, cat_cols, num_cols)

ebill_status_desc ======================
Before :  ['Ebill Only' 'Not Ebill' 'Ebill And Print Statement' 'None']
After  :  ['Ebill' 'Not Ebill' 'None']

agent_last_dept ======================
Before :  ['None' 'Customer Service Rep' 'Technical Service Rep' 'Retention L1'
 'Retention L2 SBR' 'Sales' 'Retention MT' 'Retention L2 ESC'
 'Shared Services' 'Sales Commercial' 'Retention BPO'
 'Retention Commercial' 'Ecommerce Sales' 'West Sales Support'
 'Sales Support']
After  :  ['None' 'Customer Service Rep' 'Technical Service Rep' 'Retention' 'Sales'
 'Shared Services']

 ivr_last_dept ======================
Before :  ['None' 'Customer Service Rep' 'Technical Service Rep' 'Retention L1'
 'Retention L2 SBR' 'Sales' 'Retention MT' 'Retention L2 ESC' 'Retention'
 'Retention Escalation' 'Unknown' 'Retention BPO' 'Sales Commercial'
 'Shared Services' 'Retention Commercial' 'Sales Support']
After  :  ['None' 'Customer Service Rep' 'Technical Service Rep' 'Retention' 'Sales'
 'Shared Services']


In [28]:
cat_cols_step1 = dict1['final_cat_cols']
num_cols_step1 = dict1['final_num_cols']

print('categorical columns : ', len(cat_cols_step1))
print('numerical   columns : ', len(num_cols_step1))

raw_data_res_cpy_step1 = dict1['df']
print('shape : ', raw_data_res_cpy_step1.shape)

46
97


(2870765, 144)

In [30]:
raw_data_res_cpy_step1.to_pickle('Scoring_pickles/raw_data_res_cpy_step1.pkl')

raw_data_res_cpy_step1 = pd.read_pickle("Scoring_pickles/raw_data_res_cpy_step1.pkl")  
raw_data_res_cpy_step1.info()

file_name1 = "Scoring_pickles/cat_cols_step1.pkl"

open_file1 = open(file_name1, "wb")
pickle.dump(cat_cols_step1, open_file1)
open_file1.close()

# open_file1 = open(file_name1, "rb")
# cat_cols_jul22_step1 = pickle.load(open_file1)
# open_file1.close()

file_name2 = "Scoring_pickles/num_cols_step1.pkl"

open_file2 = open(file_name2, "wb")
pickle.dump(num_cols_step1, open_file2)
open_file2.close()

# open_file2 = open(file_name2, "rb")
# num_cols_jul22_step1 = pickle.load(open_file2)
# open_file2.close()

### Step2 : Rare Category encoding Categorical columns

In [ ]:
raw_data_res_cpy_step2_train = pd.read_pickle("raw_data_res_cpy_step2.pkl")  
raw_data_res_cpy_step2_train.info()

In [33]:
features_rarecat_enc = ['head_end_desc','ecohort_code_desc']

raw_data_res_cpy_step2 = raw_data_res_cpy_step1.copy()

# Keep only those levels which are used for training. Replace everything else as 'ALL_OTHER'
for col in features_rarecat_enc:
    lst = list(raw_data_res_cpy_step2_train[col].unique())
    raw_data_res_cpy_step2.loc[~(raw_data_res_cpy_step2[col].isin(lst)), col] = 'ALL_OTHER' 
    
print('shape : ', raw_data_res_cpy_step2.shape)    

In [35]:
raw_data_res_cpy_step2[features_rarecat_enc].describe()

,head_end_desc,ecohort_code_desc
count,2870765,2870765
unique,21,21
top,ALL_OTHER,ALL_OTHER
freq,1398113,1280777


### Step3: Preprocess numerical columns

##### 1. Replace max values with upper_quant percentile and min value with lower_quant percentile
##### 2. Replace negative values with 0
##### 3. Remove features with constatnt values 
##### 4. Replace missing values with 0

In [38]:
dict2 = cpp.preprocess_num_cols(raw_data_res_cpy_step2, num_cols_step1, replace_neg_values = True, remove_constant_features = False, lower_quant = 0, upper_quant = 0.995)

1. Replaced max values with lower_quant and upper_quant percentiles
2. Replaced negative values with 0
4. Replaced missing values with 0


In [39]:
num_cols_step3 = dict2['num_cols_remain']
const_cols_removed_step3 = dict2['const_cols_removed']

print(len(num_cols_step3))
print(len(const_cols_removed_step3))

raw_data_res_cpy_step3 = dict2['df']
raw_data_res_cpy_step3.shape

97
0


(2870765, 144)

In [40]:
# No missing values found
raw_data_res_cpy_step3[list(set(num_cols_step3) - set(const_cols_removed_step3))].isnull().sum()

flag_altice_DSL                         0
agent_offered_30day                     0
agent_repeat_calls_30day                0
agent_abandoned_standard_7day           0
servicevisit_once                       0
flag_altice_CABLE                       0
flag_comp_DOCSIS_3_1                    0
agent_abandoned_30day                   0
agent_offered_7day                      0
acu_ool_usage_down_mb_m1                0
bill_m4                                 0
servicevisit_threetimes                 0
agent_abandoned_7day                    0
service_visits_7day                     0
FAILSTAT_WIFI_QOE_HOME_30               0
ool_offer_group_v2_desc_m2              0
acu_ool_usage_up_mb_m2                  0
ool_offer_group_v2_desc_m0              0
ivr_repeat_calls_30day                  0
roll_off_lift_m2                        0
ov_offer_group_v2_desc_m1               0
rateevent_baserevchange                 0
service_visit_1_tc_fix_code_desc        0
totalcontact_m2                   

### Step4: One hot encoding of Categorical columns

In [43]:
fit_ohe   = cpp.UniversalCategoryEncoder(encoding_method='ohe').fit(X = raw_data_res_cpy_step3, features_CAT_ = cat_cols_step1)
dict3     = fit_ohe.transform(X = raw_data_res_cpy_step3)

X_ohe_step4 = dict3['X_transformed']
print('Onehot encoded columns dataframe shape : ', X_ohe_step4.shape)

raw_data_res_cpy_step4 = dict3['X_full']
print('Full final dataframe shape             : ', raw_data_res_cpy_step4.shape)

In [47]:
# raw_data_res_cpy_step4.to_pickle('raw_data_res_cpy_step4.pkl')

### Prediction

In [2]:
# raw_data_res_cpy_step4 = pd.read_pickle("raw_data_res_cpy_step4.pkl")  

In [3]:
raw_data_res_cpy_step4.set_index('chc_id', inplace= True)

In [4]:
## Check missing values
df_missing = pd.DataFrame(raw_data_res_cpy_step4.isnull().sum(), columns = ['missing']) #/raw_data_res_v2[num_cols_v2].shape[0]
df_missing[df_missing['missing'] > 0]

,missing


In [5]:
raw_data_res_cpy_step4.columns = raw_data_res_cpy_step4.columns.str.split('.').str[0]
raw_data_res_cpy_step4.columns

Index(['agentlastdep_WIFIRebootdistress_None_0',
       'agentlastdep_WIFIRebootdistress_None_1',
       'agentlastdep_WIFIRebootdistress_Customer Service Rep_0',
       'agentlastdep_WIFIRebootdistress_Technical Service Rep_0',
       'agentlastdep_WIFIRebootdistress_Retention_0',
       'agentlastdep_WIFIRebootdistress_Technical Service Rep_1',
       'agentlastdep_WIFIRebootdistress_Customer Service Rep_1',
       'agentlastdep_WIFIRebootdistress_Sales_0',
       'agentlastdep_WIFIRebootdistress_Retention_1',
       'agentlastdep_WIFIRebootdistress_Sales_1',
       ...
       'flag_comp_TERRESTRIAL_FIXED', 'ov_tier_change_m4m0',
       'rateevent_baserevchange', 'agentcall_and_repeat1',
       'agentcall_and_repeat2', 'agentcall_and_repeat3', 'servicevisit_once',
       'servicevisit_twotimes', 'servicevisit_threetimes',
       'agentcall_servicevisit'],
      dtype='object', length=513)

In [6]:
filename = 'fixed_churn_best_model_top215_features_EAST.sav'
best_model = pickle.load(open(filename, 'rb'))

In [7]:
feature_importances_df = pd.read_csv('best_model_feature_importances_EAST.csv')  
feature_importances_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          498 non-null    int64  
 1   feature             498 non-null    object 
 2   Importance          498 non-null    float64
 3   ranking_importance  498 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 15.7+ KB


In [8]:
best_nb = 215 # top 215 features gave 70% information
best_features = list(feature_importances_df[0:best_nb]['feature'])
len(best_features)

215

In [9]:
best_features

['tenure_productclass_morethan_60mon_VIDEO/OOL/OV',
 'flag_comp_FIBER',
 'speed_m0_50',
 'head_end_desc_BRIDGEPORT',
 'ivr_last_ivrInteractionEndResult_Transfer',
 'speed_m0_ALL_OTHER',
 'cust_tenure_morethan_60mon',
 'cust_equip_class_speedm0_LEGACY_50',
 'local_market_segment_LIE',
 'oolpromo_rng_m0_a: 0-2',
 'head_end_desc_CORAM',
 'local_market_segment_North Atlantic',
 'curr_video_tier_desc2_m0_SELECT',
 'promo_rolloff_count_60day',
 'flag_comp_DOCSIS_3_1',
 'curr_video_tier_desc2_m0_BASIC',
 'hbo_svod_change_m3m1_same',
 'totalcontact_m0',
 'head_end_desc_OAKLAND',
 'local_market_segment_CT',
 'hbo_svod_change_m3m1_down',
 'flag_altice_DOCSIS_3_1',
 'agent_repeat_3_prevInteractionReason_Data',
 'curr_video_tier_desc2_m0_VALUE',
 'agent_accepted_7day',
 'service_visit_1_tc_fix_code_desc',
 'cust_equip_class_speedm0_LEGACY_ALL_OTHER',
 'curr_video_tier_desc2_m0_PREMIER',
 'tenure_productclass_0_6mon_OOL',
 'iptel_revenue',
 'tenure_income_morethan_60mon_Elite',
 'ivr_last_interacti

In [10]:
raw_data_res_jul22_cpy_step4[best_features].head(5)

,tenure_productclass_morethan_60mon_VIDEO/OOL/OV,flag_comp_FIBER,speed_m0_50,head_end_desc_BRIDGEPORT,ivr_last_ivrInteractionEndResult_Transfer,speed_m0_ALL_OTHER,cust_tenure_morethan_60mon,cust_equip_class_speedm0_LEGACY_50,local_market_segment_LIE,oolpromo_rng_m0_a: 0-2,head_end_desc_CORAM,local_market_segment_North Atlantic,curr_video_tier_desc2_m0_SELECT,promo_rolloff_count_60day,flag_comp_DOCSIS_3_1,curr_video_tier_desc2_m0_BASIC,hbo_svod_change_m3m1_same,totalcontact_m0,head_end_desc_OAKLAND,local_market_segment_CT,hbo_svod_change_m3m1_down,flag_altice_DOCSIS_3_1,agent_repeat_3_prevInteractionReason_Data,curr_video_tier_desc2_m0_VALUE,agent_accepted_7day,service_visit_1_tc_fix_code_desc,cust_equip_class_speedm0_LEGACY_ALL_OTHER,curr_video_tier_desc2_m0_PREMIER,tenure_productclass_0_6mon_OOL,iptel_revenue,tenure_income_morethan_60mon_Elite,ivr_last_interactionReason_Retention,head_end_desc_CENTRAL ISLIP HAUPPAUGE,local_market_segment_LIW,agent_offered_7day,head_end_desc_ST,speed_m0_30,archetype_desc_Movers and Shakers,ool_offer_group_v2_desc_m4,curr_video_tier_desc2_m0_ALL_OTHER,agentlastdep_WIFIRebootdistress_None_0,analog_revenue,ivr_last_dept_Retention,agent_accepted_30day,cust_tenure_24_60mon,curr_video_tier_desc2_m0_None,ecohort_code_desc_Affluent Elders: Older Upscale Suburbanites,video_tier_change_m4m0_OTHER,oolpromo_rng_m0_c: 6-11,totalcontact_m1,tenure_productclass_morethan_60mon_OV,flag_comp_DSL,ivr_repeat_3_prevDept_ALL_OTHER,cust_equip_class_speedm0_LEGACY_30,tenure_income_morethan_60mon_High,FAILSTAT_OFFLINEEVT,hbo_svod_change_m3m1_up,cust_equip_class_speedm0_FIBER GATEWAY_300,ivr_last_ivrInteractionEndResult_Disconnect,head_end_desc_RANDOLPH,ivr_repeat_1_prevInteractionReason_General,ool_offer_group_v2_desc_m1,product_class_desc_OV,archetype_desc_Indulgent Empty Nesters,tenure_income_24_60mon_Not Assigned,agentlastdep_OFFLINEdistress_None_0,speed_m0_25,cust_tenure_0_6mon,head_end_desc_HICKSVILLE,ivr_last_dept_Sales,head_end_desc_STAMFORD,curr_video_tier_desc2_m0_ECONOMY,agent_offered_30day,ool_offer_group_v2_desc_m2,avg_sports_sc_rev_m1_m6,service_visit_2_tc_fix_code_desc,agent_repeat_3_prevDept_None,stz_enc_svod_change_m3m1_up,local_market_segment_BROOKLYN,ecohort_income_range_desc_Low,ecohort_spending_type_desc_Very High,head_end_desc_BRONX - HUB E (TIFFANY),roll_off_lift_m0,ool_offer_group_v2_desc_m0,tenure_income_24_60mon_High,agent_last_dept_None,ivr_repeat_3_prevInteractionReason_General,tenure_productclass_morethan_60mon_VIDEO/OOL,ecohort_code_desc_Planners and Savers: City Couples,cust_equip_class_speedm0_FIBER GATEWAY_500,oolpromo_rng_m0_d: 12-13,tenure_productclass_6_12mon_VIDEO/OOL/OV,agentlastdep_OFFLINEdistress_Sales_1,service_visit_2_tc_problem_code_desc,ivr_repeat_1_prevInteractionReason_Sales,agentlastdep_WIFIQoEdistress_Retention_0,cust_equip_class_speedm0_ALTICE GATEWAY_400,ecohort_credit_line_type_desc_Not Assigned,ovpromo_rng_m0_b: 3-5,ivr_repeat_2_prevInteractionReason_Data,videopromo_rng_m0_a: 0-2,curr_video_tier_desc2_m0_CORE,tenure_productclass_24_60mon_OOL/OV,head_end_desc_BROOKLYN - HUB L (FULTON),cust_equip_class_speedm0_FIBER GATEWAY_1000,stz_enc_svod_change_m3m1_same,ecohort_code_desc_Suburban Success: Upscale Families,agent_repeat_3_prevDept_Customer Service Rep,speed_m0_100,videopromo_rng_m0_d: 12-13,ivr_repeat_2_prevInteractionReason_General,agentlastdep_WIFIRebootdistress_Sales_0,service_visit_1_tc_problem_code_desc,local_market_segment_SOUTH NJ,tenure_productclass_morethan_60mon_OOL/OV,head_end_desc_NORTH BERGEN (HUDSON SYSTEM),head_end_desc_BROOKLYN - HUB K (60TH ST,tenure_income_0_6mon_Elite,flag_altice_DOCSIS_3_0,ecohort_spending_type_desc_Very Low,tenure_income_0_6mon_Low,ecohort_credit_usage_type_desc_Very High,local_market_segment_WC,agentlastdep_DHCPdistress_Sales_1,ecohort_code_desc_Comfortable with Credit: Upscale Retirees,ecohort_code_desc_Comfortable Retirement: Suburban Couples,cust_equip_class_FIBER GATEWAY,rate_event_count_60day,ivr_repeat_3_prevInteractionReas

In [13]:
# Predict Data
y_pred  = best_model.predict(raw_data_res_cpy_step4[best_features])
y_proba = best_model.predict_proba(raw_data_res_cpy_step4[best_features])[:,1]

In [14]:
raw_data_res_cpy_step4.index

Int64Index([787653587901, 780151673907, 781910935204, 784415074711,
            780129568409, 783600176805, 787519338803, 784419011207,
            787516836913, 786710078208,
            ...
            785886191809, 780110911809, 787018704102, 786990853505,
            786711215301, 783661321803, 785888811205, 788353919405,
            787512575908, 786828514311],
           dtype='int64', name='chc_id', length=2870765)

In [15]:
## Build final data
df_predict_actual                   = pd.DataFrame()
df_predict_actual['predict']        = y_pred
df_predict_actual['predict_prob']   = y_proba
df_predict_actual['chc_id']         = raw_data_res_cpy_step4.index

df_predict_actual.head()

,predict,predict_prob,chc_id
0,0,0.478901,787653587901
1,0,0.423671,780151673907
2,0,0.455785,781910935204
3,1,0.573429,784415074711
4,1,0.576973,780129568409


In [22]:
df_fixed    = df_predict_actual.sort_values(by = 'predict_prob', ascending = False).reset_index()
sample_size = df_predict_actual.shape[0]

# Segment the data into Decile size of 1
decile_size = sample_size/100 
df_fixed['Rank'] = ((df_predict_actual.index//decile_size)*1+1).astype('int64')

df_fixed2 = df_fixed[df_fixed['Rank'] <= 20][['chc_id', 'Rank']]
df_fixed2.head()

##### Append additional filters that business asked for

In [3]:
# raw_data_res_cpy_step3 = pd.read_pickle("raw_data_res_cpy_step3.pkl")  
# raw_data_res_cpy_step3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2870765 entries, 2 to 4925203
Columns: 144 entries, head_end_desc to agentlastdep_WIFIQoEdistress
dtypes: float64(97), int64(1), object(44), string(2)
memory usage: 3.1+ GB


In [2]:
# raw_data_res_jul22_OPT = pd.read_pickle("Data/OPT_Scoring/Jul22/raw_data_res_jul22_OPT.pkl")  
# raw_data_res_jul22_OPT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2870765 entries, 2 to 4925203
Columns: 150 entries, head_end_desc to chc_id
dtypes: float64(56), int64(1), object(93)
memory usage: 3.2+ GB


In [4]:
z = pd.merge(left = df_fixed2, right = raw_data_res, how = 'inner', left_on = 'chc_id', right_on = 'chc_id')
z.shape

(574154, 151)

In [5]:
z['cust_tenure_days'] = z['cust_tenure_days'].astype('float64')
z.loc[(z['cust_tenure_days'] <= 182), 'cust_tenure'] = '0_6mon'
z.loc[(z['cust_tenure_days'] > 182) & (z['cust_tenure_days'] <= 365), 'cust_tenure'] = '6_12mon'
z.loc[(z['cust_tenure_days'] > 365) & (z['cust_tenure_days'] <= 730), 'cust_tenure'] = '12_24mon'
z.loc[(z['cust_tenure_days'] > 730) & (z['cust_tenure_days'] <= 1825), 'cust_tenure'] = '24_60mon'
z.loc[(z['cust_tenure_days'] > 1825), 'cust_tenure'] = 'morethan_60mon'

In [6]:
z['flag_comp_COAX'] = np.where((z["flag_comp_CABLE"] == 1) | (z["flag_comp_DSL"] == 1) | (z["flag_comp_OTHER"] == 1) | (z["flag_comp_TERRESTRIAL_FIXED"] == 1), 1, 0)

z["flag_comp_FIBER"]      = np.where((z["flag_comp_FIBER"] == 1), 'FIBER', 'NONE')
z["flag_comp_DOCSIS_3_0"] = np.where((z["flag_comp_DOCSIS_3_0"] == 1), 'DOCSIS_3_0', 'NONE')
z["flag_comp_DOCSIS_3_1"] = np.where((z["flag_comp_DOCSIS_3_1"] == 1), 'DOCSIS_3_1', 'NONE')
z["flag_comp_COAX"]       = np.where((z["flag_comp_COAX"] == 1), 'COAX', 'NONE')

z['competition'] = z['flag_comp_FIBER'] + ',' + z['flag_comp_DOCSIS_3_0'] + ',' + z['flag_comp_DOCSIS_3_1'] + ',' + z['flag_comp_COAX']

z['competition'] = z['competition'].str.replace(',NONE','')
z['competition'] = z['competition'].str.replace('NONE,','')

z.head()

,chc_id,Rank,head_end_desc,local_market_segment,metro_area_desc,product_class_desc,cust_equip_class,ecohort_code_desc,ecohort_income_range_desc,ecohort_age_range_desc,ecohort_spending_type_desc,ecohort_credit_line_type_desc,ecohort_credit_usage_type_desc,archetype_desc,ebill_status_desc,agent_last_dept,agent_repeat_1_prevDept,agent_repeat_1_prevInteractionReason,agent_repeat_2_prevDept,agent_repeat_2_prevInteractionReason,agent_repeat_3_prevInteractionReason,agent_repeat_3_prevDept,ivr_last_dept,ivr_last_interactionReason,ivr_last_ivrInteractionEndResult,ivr_repeat_1_prevDept,ivr_repeat_1_prevInteractionReason,ivr_repeat_2_prevDept,ivr_repeat_2_prevInteractionReason,ivr_repeat_3_prevDept,ivr_repeat_3_prevInteractionReason,speed_m0,speed_m1,speed_m2,speed_m3,speed_m4,curr_video_tier_desc2_m0,curr_video_tier_desc2_m1,curr_video_tier_desc2_m2,curr_video_tier_desc2_m3,curr_video_tier_desc2_m4,curr_ov_tier_desc2_m0,curr_ov_tier_desc2_m1,curr_ov_tier_desc2_m2,curr_ov_tier_desc2_m3,curr_ov_tier_desc2_m4,hbo_svod_m1,max_svod_m2,max_svod_m3,ool_offer_group_v2_desc_m0,ool_offer_group_v2_desc_m1,ool_offer_group_v2_desc_m2,ool_offer_group_v2_desc_m3,ool_offer_group_v2_desc_m4,oolpromo_rng_m0,ov_offer_group_v2_desc_m0,ov_offer_group_v2_desc_m1,ov_offer_group_v2_desc_m2,ov_offer_group_v2_desc_m3,ov_offer_group_v2_desc_m4,ovpromo_rng_m0,stz_enc_svod_m1,stz_enc_svod_m2,stz_enc_svod_m3,video_offer_group_desc_m0,video_offer_group_desc_m1,video_offer_group_desc_m2,video_offer_group_desc_m3,video_offer_group_desc_m4,service_visit_1_tc_problem_code_desc,service_visit_1_tc_fix_code_desc,service_visit_2_tc_problem_code_desc,service_visit_2_tc_fix_code_desc,service_visit_3_tc_problem_code_desc,service_visit_3_tc_fix_code_desc,videopromo_rng_m0,analog_revenue,iptel_revenue,ool_revenue,dvr_revenue,auto_ind,agent_abandoned_standard_7day,agent_abandoned_7day,agent_offered_7day,agent_accepted_7day,agent_abandoned_standard_30day,agent_abandoned_30day,agent_offered_30day,agent_accepted_30day,agent_repeat_calls_30day,ivr_calls_7day,ivr_calls_30day,ivr_repeat_calls_30day,service_visits_7day,service_visits_30day,service_visits_90day,outages_90day,outages_60day,outages_30day,rate_event_count_60day,promo_rolloff_count_60day,FAILSTAT_OFFLINEEVT,FAILSTAT_DHCP_MACADDR,FAILSTAT_WIFI_REBOOT_COUNT,FAILSTAT_WIFI_QOE_HOME_30,acu_ool_usage_down_mb_m1,acu_ool_usage_up_mb_m1,acu_ool_usage_down_mb_m2,acu_ool_usage_up_mb_m2,avg_sports_sc_rev_m1_m6,baserev_m0,baserev_m1,baserev_m2,baserev_m3,baserev_m4,baserev_m5,baserev_m6,bill_m0,bill_m1,bill_m2,bill_m3,bill_m4,bill_m5,bill_m6,mb_total_usage1,mb_total_usage2,roll_off_lift_m0,roll_off_lift_m1,roll_off_lift_m2,roll_off_lift_m3,roll_off_lift_m4,totalcontact_m0,totalcontact_m1,totalcontact_m2,totalcontact_m3,totalcontact_m4,flag_altice_CABLE,flag_altice_DOCSIS_3_0,flag_altice_DOCSIS_3_1,flag_altice_DSL,flag_altice_FIBER,flag_altice_OTHER,flag_altice_TERRESTRIAL_FIXED,flag_comp_CABLE,flag_comp_DOCSIS_3_0,flag_comp_DOCSIS_3_1,flag_comp_DSL,flag_comp_FIBER,flag_comp_OTHER,flag_comp_TERRESTRIAL_FIXED,cust_tenure_days,cust_tenure,flag_comp_COAX
0,781800510402,1,CENTRAL ISLIP HAUPPAUGE,LIW,NYMA,VIDEO/OOL/OV,ALTICE GATEWAY,Careers First: Urbanites,Elite,Working Years,High,High,Low,Busy Families Achievers,Not Ebill,Customer Service Rep,Customer Service Rep,Billing,Technical Service Rep,Data,Video,Technical Service Rep,Customer Service Rep,General,Transfer,Retention L1,Retention,Technical Service Rep,Data,Sales,Sales,OPTIMUM 1GBPS-COAX,OPTIMUM 1GBPS-COAX,OPTIMUM 1GBPS-COAX,OPTIMUM 1GBPS-COAX,OPTIMUM 1GBPS-COAX,OPTIMUM PREMIER,OPTIMUM PREMIER,OPTIMUM PREMIER,OPTIMUM PREMIER,OPTIMUM PREMIER,$10,$10,$10,$10,$10,Y,N,N,Video/OOL Promo-12,Video/OOL Promo-12,Video/OOL Promo-12,Video/OOL Promo-12,Video/OOL Promo-12,b: 3-5,OV - $10.00 Promo,OV - $10.00 Promo,OV - $10.00 Promo,OV - $10.00 Promo,OV - $10.00 Promo,None,Y,Y,Y,Video/OOL Promo,Video/OOL Promo,Video/OOL Promo,Video/OOL Promo,Video/OOL Promo,113-TV | INTERMT|1-OTLTS,138-SWAP ALTICE ONE 

In [10]:
z['competition'].unique()

array(['FIBER,COAX', 'FIBER', 'NONE', 'COAX', 'FIBER,DOCSIS_3_0,COAX',
       'FIBER,DOCSIS_3_1,COAX', 'DOCSIS_3_1,COAX', 'DOCSIS_3_0,COAX',
       'FIBER,DOCSIS_3_1'], dtype=object)

In [11]:
z.rename(columns = {'baserev_m0'                :'baserev_current_month',
                    'FAILSTAT_OFFLINEEVT'       :'mdm_offline_freq_30_days',
                    'FAILSTAT_DHCP_MACADDR'     :'DHCP_flapping_freq_30_days',
                    'FAILSTAT_WIFI_REBOOT_COUNT':'wifi_reboot_freq_30_days',
                    'FAILSTAT_WIFI_QOE_HOME_30' :'wifi_QOE_freq_30_days',
                    'agent_offered_30day'       :'call_offered_30day_prior'}, inplace = True)

cols = ['chc_id', 'Rank', 'local_market_segment', 'archetype_desc', 'baserev_current_month', 'rate_event_count_60day', \
   'cust_equip_class','product_class_desc', \
   'mdm_offline_freq_30_days', 'DHCP_flapping_freq_30_days', 'wifi_reboot_freq_30_days', 'wifi_QOE_freq_30_days', \
   'competition', 'call_offered_30day_prior','cust_tenure']

z[cols].info()

In [18]:
z[cols] = z[cols].fillna(0)
z[cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 574154 entries, 0 to 574153
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   chc_id                      574154 non-null  int64  
 1   Rank                        574154 non-null  int64  
 2   local_market_segment        574154 non-null  object 
 3   archetype_desc              574154 non-null  object 
 4   baserev_current_month       574154 non-null  object 
 5   rate_event_count_60day      574154 non-null  float64
 6   cust_equip_class            574154 non-null  object 
 7   product_class_desc          574154 non-null  object 
 8   30_days_mdm_offline_freq    574154 non-null  float64
 9   30_days_DHCP_flapping_freq  574154 non-null  float64
 10  30_days_wifi_reboot_freq    574154 non-null  float64
 11  30_days_wifi_QOE_freq       574154 non-null  float64
 12  competition                 574154 non-null  object 
 13  call_offered_3

In [14]:
df_ret_callrate = pd.read_csv('retention_callrate_from top20pctl_customers_EAST.csv')
df_ret_callrate.head()

,Unnamed: 0,Rank,calls_feb22,cust_feb22,call_rate_feb22,calls_nov21,cust_nov21,call_rate_nov21,retention_call_rate
0,0,1,10303,29136,0.353618,5928,29269,0.202535,0.278
1,1,2,8133,29136,0.279139,4485,29268,0.153239,0.216
2,2,3,7061,29136,0.242346,3885,29268,0.132739,0.188
3,3,4,6754,29136,0.231809,3469,29268,0.118525,0.175
4,4,5,6311,29135,0.216612,3157,29268,0.107865,0.162


In [20]:
final_scored_list = z.merge(df_ret_callrate, on='Rank',how='left') 

final_scored_list[['chc_id', 'Rank', 'local_market_segment', 'archetype_desc', 'baserev_current_month', 'rate_event_count_60day', \
   'cust_equip_class','product_class_desc', \
   '30_days_mdm_offline_freq', '30_days_DHCP_flapping_freq', '30_days_wifi_reboot_freq', '30_days_wifi_QOE_freq', \
   'competition', \
   'call_offered_30day_prior','cust_tenure', 'retention_callrate']].to_csv("Aug22_scoring_top20_percentile_EAST.csv",index=False)